## CHAPTER15.클러스터에서 스파크 실행하기

이 장에서는 스파크에서 코드를 실행할 때 어떤 일이 발생하는지 알아본다.

### 15.1 스파크 애플리케이션의 아키텍처

![cluster-overview](../../assets/presentations/week15/cluster-overview.png)

![structure_of_a_spark_application](../../assets/presentations/week15/structure_of_a_spark_application.png)


- 스파크 드라이버
    - 작업을 관리
    - `스파크 컨텍스트` 객체를 생성하여 클러스터 매니저와 통신하면서 클러스터의 자원 관리를 지원하고 애플리케이션의 라이프 사이클을 관리한다.
    - 스파크 애플리케이션의 실행을 제어하고, 스파크 클러스터(익스큐터의 상태와 태스크)의 모든 상태 정보를 유지한다.
    - 물리적 컴퓨팅 자원 확보와 익스큐터 실행을 위해 클러스터 매니저와 통신할 수 있다.
- 스파크 익스큐터
    - 태스크를 수행하는 프로세스
    - 스파크 드라이버가 할당한 태스크를 받아 실행하고 태스크의 상태와 결과를 드라이버에 보고한다.
    - 모든 스파크 애플리케이션은 개별 익스큐터 프로세스를 사용한다.
    - https://jaemunbro.medium.com/spark-executor-%EA%B0%9C%EC%88%98-%EC%A0%95%ED%95%98%EA%B8%B0-b9f0e0cc1fd8

- 클러스터 매니저
    - 작업이 실행되는 노드를 관리
    - 스파크 애플리케이션의 실행 과정에서 애플리케이션이 실행되는 머신을 관리한다. 즉 모든 스파크 애플리케이션과 관련된 프로세스를 유지하는 역할을 한다. 스파크 어플리케이션의 리소스를 효율적으로 배분한다.
    - 클러스터 매니저는 '드라이버'와 '워커'라는 개념을 가지고 있으며, 혼란스러울 수 있지만 가장 큰 차이점은 프로세스가 아닌 물리적인 머신에 연결되는 점이다.
    - 클러스터 매니저는 드라이버 노드랑 개별 워커 노드이며, 원은 개별 워커 노드를 실행하고 관리하는 데몬 프로세스이다.
    ![clusterdriver_worker](../../assets/presentations/week15/clusterdriver_worker.png)
    ** 그림에서 스파크 애플리케이션은 아직 실행되지 않았고, 원은 클러스터 매니저의 프로세스일 뿐이다.

    - 스파크 애플리케이션을 실제로 실행할 때가 되면 클러스터 매니저에 자원 할당을 요청한다. 사용자 애플리케이션의 설정에 따라 스파크 드라이버를 실행할 자원을 포함해 요청하거나 스파크 애플리케이션 실행을 위한 익스큐터 자원을 요청할 수도 있다. 

    - 스파크가 지원하는 클러스터 매니저
        - standalone(스탠드얼론 클러스터 매니저)
            - spark에 포함된 간단한 클러스터 관리자로 클러스터를 쉽게 설정할 수 있다.
        - Apache Mesos(아파치 메소스)
            - Hadoop MapReduce 및 서비스 애플리케이션도 실행할 수 있는 일반 클러스터 관리자이다. (더 이상 사용되지 않음)
        - Hadoop YARN(하둡 YARN)
            - Hadoop 2 및 3의 리소스 관리자
        - kubernetes
            - 컨테이너화된 애플리케이션의 배포, 확장 및 관리를 자동화하기 위한 오픈 소스 시스템

참고: https://wikidocs.net/26630

### 15.1.1 실행 모드
실행 모드는 애플리케이션을 실행할 때 요청한 자원의 물리적인 위치를 결정한다. 
- 실행 모드
    - 클러스터 모드
    - 클라이언트 모드
    - 로컬 모드

** 실선 직사각형: 스파크 드라이버 프로세스
** 점선 직사각형: 익스큐터 프로세스

**클러스터 모드**
- 클러스터 모드는 가장 흔하게 사용되는 스파크 애플리케이션 실행 방식
- 클러스터 모드를 사용하려면 컴파일된 JAR 파일, 파이썬 스크립트를 클러스터 매니저에 전달해야 한다
- 클러스터 매니저는 파일을 받은 다음 워커 노드에 드라이버와 익스큐터 프로세스를 실행한다

![spark's_cluster_mode](../../assets/presentations/week15/spark's_cluster_mode.png)
    - 하나의 워커 노드에 스파크 드라이버를 할당하고 다른 워커 노드에 익스큐터를 할당하는 모습


**클라이언트 모드**
- 애플리케이션을 제출한 클라이언트 머신에 스파크 드라이버가 위치한다
- 클라이언트 머신은 스파크 드라이버 프로세스를 유지하며 클러스터 매니저는 익스큐터 프로세스를 유지한다.
    - 드라이버는 클러스터 외부 머신에서 실행
    - 나머지 워커는 클러스터에 위치
- 스파크 애플리케이션이 클러스터와 무관한 머신에서 동작하는데, 보통 이런 머신을 게이트웨이 머신 또는 에지 노드라고 부른다.

![spark's_cluster_mode2](../../assets/presentations/week15/spark's_cluster_mode2.png)


**로컬 모드**
- 모든 스파크 애플리케이션은 단일 머신에서 실행
- 애플리케이션 병렬 처리를 위해 단일 머신의 스레드를 활용한다.
- 스파크 학습 또는 애플리케이션 테스트에서 주로 사용하고 운영용 애플리케이션을 실행할 때는 사용을 권장하지 않는다.


### 15.2 스파크 애플리케이션의 생애주기(스파크 외부)
- 하나의 드라이버 노드와 세 개의 워커 노드로 구성된 총 네 대 규모의 클러스터가 이미 실행되고 있다고 가정 (**이 시점에서 클러스터 매니저는 중요하지 않음)

### 15.2.1 클라이언트 요청
- 스파크 애플리케이션(컴파일된 JAR나 라이브러리 파일을 의미)을 제출한다.
- 스파크 애플리케이션을 제출하는 시점에 로컬 머신에서 코드가 실행되어 클러스터 드라이버 노드에 요청한다.
    - 이 과정에서 스파크 드라이버 프로세스의 자원을 함께 요청한다.
- 클러스터 매니저는 이 요청을 받아들이고 노드 중 하나에 드라이버 프로세스를 실행한다.
- 스파크 잡을 제출한 클라이언트 프로세스는 종료되고 애플리케이션은 클러스터에서 실행된다.

![Requesting_resources_for_a_driver.png](../../assets/presentations/week15/Requesting_resources_for_a_driver.png)


### 15.2.2 시작
![lanching_the_spark_application](../../assets/presentations/week15/lanching_the_spark_application.png)
- 드라이버 프로세스가 클러스터에 배치되었으므로 사용자 코드를 실행할 차례
- 사용자 코드에는 반드시 스파크 클러스터(드라이버와 익스큐터)를 초기화하는 SparkSession이 포함되어야 한다.
- SparkSession은 클러스터 매니저와 통신(어두운 선)해 스파크 익스큐터 프로세스의 실행(밝은선)을 요청한다.
- 클러스터 매니저는 익스큐터 프로세스를 시작하고 응답받아 익스큐터의 위치와 관련 정보를 드라이버 프로세스로 전송한다.
- 모든 작업이 정상적으로 완료되면 스파크 클러스터가 완성된다.

### 15.2.3 실행
![execution_application](../../assets/presentations/week15/execution_application.png)
- 스파크 클러스터가 생성되었으므로 코드를 실행한다.
- 드라이버와 워커는 코드를 실행하고 데이터를 이동하는 과정에서 서로 통신한다.
    - 드라이버는 각 워커에 태스크를 할당
    - 태스크를 할당받은 워커는 태스크의 상태와 성공/실패 여부를 드라이버에 전송 

### 15.2.4 완료
![shutting_down_the_application](../../assets/presentations/week15/shutting_down_the_application.png)
- 스파크 애플리케이션의 실행이 완료되면 드라이버 프로세스가 성공이나 실패 중 하나의 상태로 종료된다.
- 클러스터 매니저는 드라이버가 속한 스파크 클러스터의 모든 익스큐터를 종료시킨다.
- 스파크 애플리케이션의 성공/실패 여부를 컬러스터 매니저에 요청해 확인할 수 있다.






### 15.3 스파크 애플리케이션의 생애주기(스파크 내부)
애플리케이션 실행 때 스파크 내부의 동작 과정
- 스파크 애플리케이션을 정의하는 실제 '사용자 코드'와 관련
- 스파크 애플리케이션은 하나 이상의 스파크 잡으로 구성

### 15.3.1 SparkSession
- 모든 스파크 애플리케이션은 가장 먼저 SparkSession을 생성한다.
- new SparkContext 패턴을 사용하는 것보다 SparkSession의 빌더 메서드를 사용해 생성할 것을 추천
- Starting Point: SparkSession
- SparkSession을 생성하면 스파크 코드를 실행할 수 있다.
- SparkSession을 사용해 모든 저수준 API,  기존 컨텍스트 그리고 관련 설정 정보에 접근할 수 있다.


```python
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()
```


### 15.3.2 SparkContext
- SparkSession의 SparkContext는 스파크 클러스터에 대한 연결을 나타낸다.
- SparkContext로 RDD 어큐뮬레이터 그리고 브로드캐스트 변수를 생성하고 코드를 실행할 수 있다.
- 대부분의 경우 SparkSession으로 SparkContext에 접근할 수 있으므로 명시적으로 SparkContext를 초기화할 필요가 없어졌음.
- 직접 초기화하는 가장 일반적인 방법은 getOrCreate 메서드를 사용하는 것
- 과거 버전의 스파크에서는 SparkCotext와 SQLContext 두 개의 컨텍스트가 존재
    - SparkContext - 스파크의 핵심 추상화 개념을 다루는데 중점
    - SQLContext - 스파크SQL과 같은 고수준 API 기능을 다루는데 중점
    - 스파크 2.x 버전에서 두 컨텍스트를 단일 SparkSession으로 접근가능 
    - SQLContext는 사용할 필요 없으며 SparkContext 거의 사용할 필요 없음


### 15.3.3 논리적 명령
- 스파크 코드는 트랜스포메이션과 액션으로 구성
- SQL, 저수준 RDD 처리, 머신러닝 알고리즘 등을 사용해 트랜스포메이션과 액션을 마음대로 구성할 수 있음
- 선언적 명령을 사용하는 방법과 논리적 명령이 물리적 실행 계획으로 어떻게 변환되는지 이해하는 것이 중요
- 액션을 호출하면 개별 스테이지와 태스크로 이루어진 스파크 잡이 실행된다
- 로컬 머신에서 스파크 잡을 실행하면 localhost:4040으로 접속해 스파크UI를 확인할 수 있다

![sparkui1](../../assets/presentations/week15/sparkui1.png)
![sparkui2](../../assets/presentations/week15/sparkui2.png)

In [7]:
from pyspark.sql import SparkSession

# 책 예시
# spark = SparkSession.builder.master("local").appName("Word Count")\
# .config("spark.som.config.option", "some-value")\
# .getOrCreate()

# Py4JJavaError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext.
# : java.net.BindException: Can't assign requested address: Service 'sparkDriver' failed after 16 retries (on a random free port)! Consider explicitly setting the appropriate binding address for the service 'sparkDriver' (for example spark.driver.bindAddress for SparkDriver) to the correct binding address.
# 	at java.base/sun.nio.ch.Net.bind0(Native Method)
# 	at java.base/sun.nio.ch.Net.bind(Net.java:459)
# 	at java.base/sun.nio.ch.Net.bind(Net.java:448)
# 	at java.base/sun.nio.ch.ServerSocketChannelImpl.bind(ServerSocketChannelImpl.java:227)
# 	at io.netty.channel.socket.nio.NioServerSocketChannel.doBind(NioServerSocketChannel.java:134)
# 	at io.netty.channel.AbstractChannel$AbstractUnsafe.bind(AbstractChannel.java:562)

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.driver.bindAddress", "127.0.0.1") \
    .getOrCreate()

In [8]:
spark = SparkSession.builder.master("local").appName("Word Count")\
.config("spark.som.config.option", "some-value")\
.getOrCreate()

In [9]:
df1 = spark.range(2, 10000000, 2)
df2 = spark.range(2, 10000000, 4)
step1 = df1.repartition(5)
step12 = df2.repartition(6)
step2 = step1.selectExpr("id * 5 as id")
step3 = step2.join(step12, ["id"])
step4 = step3.selectExpr("sum(id)")
step4.collect()

[Row(sum(id)=2500000000000)]

In [10]:
step4.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=true
+- == Final Plan ==
   *(7) HashAggregate(keys=[], functions=[sum(id#8L)])
   +- ShuffleQueryStage 4
      +- Exchange SinglePartition, ENSURE_REQUIREMENTS, [plan_id=264]
         +- *(6) HashAggregate(keys=[], functions=[partial_sum(id#8L)])
            +- *(6) Project [id#8L]
               +- *(6) SortMergeJoin [id#8L], [id#2L], Inner
                  :- *(4) Sort [id#8L ASC NULLS FIRST], false, 0
                  :  +- AQEShuffleRead coalesced
                  :     +- ShuffleQueryStage 2
                  :        +- Exchange hashpartitioning(id#8L, 200), ENSURE_REQUIREMENTS, [plan_id=135]
                  :           +- *(3) Project [(id#0L * 5) AS id#8L]
                  :              +- ShuffleQueryStage 0
                  :                 +- Exchange RoundRobinPartitioning(5), REPARTITION_BY_NUM, [plan_id=68]
                  :                    +- *(1) Range (2, 10000000, step=2, splits=1)
                  +- *

### 15.3.3 스파크 잡
- 보통 액션 하나당 하나의 스파크 잡이 생성되며 액션은 항상 결과를 반환한다.
- 스파크 잡은 일련의 스테이지로 나뉜다.
- 스테이지 수는 셔플 작업이 얼마나 많이 발생하는지에 따라 달라진다.

### 15.3.4 스테이지
- 스파크 스테이지는 **다수의 머신에서 동일한 연산을 수행하는 태스크의 그룹**을 나타낸다
- 스파크는 가능한 많은 태스크(잡의 트랜스포메이션)를 동일한 스테이지로 묶으려 노력한다.
- **셔플 작업이 일어난 다음에는 반드시 새로운 스테이지를 시작한다**
    - 데이터의 물리적 재분배 과정
    - 예를 들어 데이터를 그룹화 -> 파티션을 재분배 하는 과정은 데이터를 이동시키는 작업이므로 익스큐터 간의 조정이 필요하다.
    - 셔플이 끝나면 새로운 스테이지를 시작
    - 최종 결과를 계산하기 위해 스테이지 실행 순서를 계속 추적한다
    
    
- <이전 잡 설명>
    - 처음 두 스테이지는 Dataframe 생성을 위한 단계
        - range 명령을 사용해 Dataframe을 생성하면 기본적으로 8개의 파티션을 생성한다.
        
    - 스테이지 3과 4: 파니션 변경
        - 데이터 셔플링으로 파티션 수 변경으로 각각 의 태스크 수에 해당하는 5개 6개 파티션으로 재분배된다.
    - 마지막 스테이지: 조인(셔플)
        - 태스크 수가 200개로 변경했다면, 스파크 SQL 설정 때문
        - spark.sql.shuffle.partitions 속성의 기본값은 200
        - spark.conf.set("spark.sql.shuffle.partitions", 50)으로 클ㄹ스터의 코어 수에 맞추어 설정 가능
        
        - 경험적으로 클러스터의 익스큐터 수보다 파티션 수를 더 크게 지정하는 것이 좋다.
        - 로컬 머신에서 코드를 실행하는 경우 병렬로 처리할 수 있는 태스크 수가 제한적이므로 이 값을 작게 설정하는 것이 좋다.
        
        
### 15.3.5 태스크
- 스파크의 스테이지는 태스크로 구성된다.
- 태스크는 데이터 단위(파티션)에 적용되는 연산 단위를 의미
- 각 태스크는 **단일 익스큐터에서 실행할 데이터의 블록**과 **다수의 트랜스포에이션 조합**으로 볼 수 있다.
    - 데이터셋이 거대한 하나의 파티션으로 구성된 경우 하나의 태스크만 생성
    - 데이터셋이 1000개의 작은 파티션으로 구성되어 있다면 1000개의 태스크를 만들어 병렬로 실행
- 파티션 수를 늘리면 더 높은 병렬성을 얻을 수 있다.


### 15.4 세부 실행 과정
- 스파크 스테이지와 태스크의 중용한 특성
- 스파크는 map 연산 후 다른 map 연산이 이어진다면 함께 실행할 수 있도록 스테이지와 태스크를 자동으로 연결
- 스파크는 모든 셔플을 작업할 때 데이터를 안정적인 저장소에 저장하므로 여러 잡에서 재사용할 수 있다.

### 15.4.1 파이프라이닝
- 스파크를 `인메모리 컴퓨팅 도구`로 만들어주는 핵심 요수 중 하나는 **스파크는 메모리나 디스크에 데이터를 쓰기 전에 최대한 많은 단계를 수행한다는 점**이다.
- 스파크가 수행하는 주요 최적화 기법 중 하나는 RDD나 RDD보다 더 아래에서 발생하는 파이프라이닝 기법이다.
- 파이프라이닝 기법
    - 노드 간의 데이터 이동 없이 각 노드가 데이터를 직접 공급할 수 있는 연산만 모아 태스크의 단일 스테이지로 만든다.
    - map, filter, map 순서로 수행되는 RDD 기반의 프로그램을 개발했다면, 개별 입력 레코드를 읽어 첫 번째 map으로 전달한 다음 filter하고 마지막 map 함수로 전달해 처리하는 과정을 태스크의 단일 스테이지로 만든다.
    - 파이프라인으로 구성된 연산 작업은 단계별로 메모리나 디스크에 중간 결과를 기록하는 방식보다 훨씬 더 처리 속도가 빠르다
    - 스파크 런타임에서 파이프라이닝을 자동을 수행하기 때문에 애플리케이션을 개발할 때는 눈에 보이지 않는다
    - 스파크 UI나 로그 파일로 애플리케이션을 호가인해보면 다수의 RDD 또는 DataFrame 연산이 하나의 스테이지로 파이프라이닝되어 있음을 알 수 있음
    
    
### 15.4.2 셔플 결과 저장
- 스파크가 reduce-by-key 연산 같이 노드 간 복제를 유발하는 연산을 실행하면 엔진에서 파이프라이닝을 수행하지 못하므로 네트워크 셔플이 발생한다.
- 노드 간 복제를 유발하는 연산
    - 항상 데이터 전송이 필요한 '소스' 태스크를 먼저 수행한다
        - 각 입력 데이터를 먼저 여러 노드로부터 복사
    - 소스 태스크의 스테이지가 실행되는 동안 셔플 파일을 로컬 디스크에 기록한다
    - 그룹화나 리듀스를 수행하는 스테이지 실행
        - 셔플 파일에서 레코드를 읽어 들인 다음 연산 수행
    - 만약 잡이 실패한 경우
        - 셔플 파일은 디스크에 저장했기 때문에 '소스' 스테이지는 재실행필요 없다. 소스스테이지가 아닌 해당 스테이지부터 처리할 수 있다.
        
- 셔플 결과 부작용
    - 이미 셔플된 데이터를 이용해 새로운 잡을 실행하면 '소스'와 관련된 셔플이 다시 실행되지 않는다는 것
    - 이미 있는 셔플 파일을 다시 사용할 수 있다고 판단한 스파크는 이전 스테이지를 처리하지 않는다. -> 스파크 UI와 로그 파일에서 'skipped'라고 표시된 사전 쇼플 스테이지를 확인할 수 있다.
    - 더 나은 성능을 얻기 위해 DataFrame이나 RDD의 cache 메서드를 사용할 수 있다. 이 경우 사용자가 직접 캐싱을 수행할 수 있어, 정확히 어떤 데이터가 어디에 저장되는지 제어할 수 있다.
